<span style='color:#009999'> <span style='font-family:serif'> <font size="15"> **SWOT Nadir NALT GDR L2**<span style='color:#0066cc'> 

<img src="img/SWOT.png" alt="drawing" width="750"/>    

<span style='color:#ff6666'><font size="5">**Requirements**
1. <font size="3"><span style='color:Black'> Have a Bearer Token for EarthData in the Cloud (See `GetStarted` Notebook).
2. <font size="3"><span style='color:Black'> Upload the Bearer Token from local file`token.json`


 <span style='color:#ff6666'><font size="5">**Objectives**
- <font size="3"><span style='color:Black'> Understand `Dimensions`, `Groups`, `Maps` and `Arrays` in the DAP4 model.
- <font size="3"><span style='color:Black'> Develop understanding of metadata of remote files accessed via OPeNDAP.
- <font size="3"><span style='color:Black'> To demonstrate a workflow for remote access and plotting of Complex (**Level 2** with Groups)  SWOT Data via OPeNDAP



<span style='color:#ff6666'><font size="5"> **Browsing Data**:

<font size="3"><span style='color:Black'> Broad information about the dataset can be found in the PODAAC website (see [here](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_NALT_GDR_2.0))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from pydap.client import open_url
import json
import cartopy.crs as ccrs

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Access EARTHDATA**

<font size="3"><span style='color:Black'> The access link can be found at [PODACC](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_NALT_GDR_2.0). This may require to be logged on to EarthDataLogin. There is data for 2023 and 2024.  



In [ ]:
data_url1 = 'https://opendap.earthdata.nasa.gov/collections/C2799438313-POCLOUD/granules/SWOT_GPR_2PfP507_010_20230501_003247_20230501_012352'
data_url2 = 'https://opendap.earthdata.nasa.gov/collections/C2799438313-POCLOUD/granules/SWOT_GPN_2PfP006_350_20231115_003412_20231115_012539'

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Import Token Authorization and create Session**
 


In [ ]:
# load token json data
with open('token.json', 'r') as fp:
    Authorization = json.load(fp)

# pass Token Authorization to a new Session.
my_session = requests.Session()
my_session.headers = Authorization


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Create dataset access via pydap**




In [ ]:
%%time
dataset1 = open_url(data_url1, session=my_session, protocol="dap4")

In [ ]:
dataset1.tree()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **This is a dataset pointing to a remote data location**


<font size="3"><span style='color:Black'> Data remains remote, no data has been downloaded.

In [ ]:
dataset1['data_01/ku/swh_ocean'].shape

In [ ]:
dataset1['data_01/time'].shape

In [ ]:
print('total array memory: ', dataset1.nbytes/1e9)

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Inspect the values**


- <font size="3"><span style='color:Black'> `longitude`
- <font size="3"><span style='color:Black'> `latitude`
- <font size="3"><span style='color:Black'> `time`



In [ ]:
dataset1['data_01/time'].attributes

In [ ]:
%%time
swh_ocean = dataset1['/data_01/ku/swh_ocean'][:] # downloads as BaseType - a thin wrapper for numpy arrays

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Maps**

<font size="3"><span style='color:Black'>  Refers to the coverage of the Satellite track. This is, how the trajectory "**maps**" with time (dimension)

In [ ]:
swh_ocean.Maps

In [ ]:
%%time
longitude1 = dataset1[swh_ocean.Maps[0]][:]
latitude1 = dataset1[swh_ocean.Maps[1]][:]

In [ ]:
longitude1.attributes

In [ ]:
latitude1.attributes



<span style='color:#ff6666'><font size="5">**CF - Conventions**

<font size="3"><span style='color:Black'> In OPeNDAP's metadata rich datasets, each contains standard attributes used to describe missing data, units in which the data is presented, and any stretching/scaling of the values. 

- <font size="3"><span style='color:Black'> `standard name`
- <font size="3"><span style='color:Black'> `units`
- <font size="3"><span style='color:Black'> `_FillValue`
- <font size="3"><span style='color:Black'> `scale_factor`
- <font size="3"><span style='color:Black'> `off_set`

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Decode Values**




In [ ]:
def decode(variable) -> np.ndarray:
    """Decodes the variable BaseType according with atributes:
        _FillValue
        scale_factor
    """
    scale_factor = 1
    _Fill_value = None

    if 'scale_factor' in variable.attributes:
        scale_factor = variable.scale_factor
    if '_FillValue' in variable.attributes:
        data = np.where(variable.data == variable._FillValue, np.nan, variable.data)    
    else:
        data = variable.data
    return scale_factor * data

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Plotting tools**

<font size="3"><span style='color:Black'> OPeNDAP does NOT include a plotting service, but OPeNDAP-served data integrates easily with plotting packages like
- <font size="3"><span style='color:Black'> Matplotlib
- <font size="3"><span style='color:Black'> Cartopy


In [ ]:
plt.figure(figsize=(15, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()
ax.stock_img() # comment this line if you do not want any background color
plt.scatter(x=decode(longitude1), y=decode(latitude1), c=decode(swh_ocean), marker='.',  cmap='jet')
plt.colorbar().set_label(swh_ocean.name + ' ['+swh_ocean.units+']')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(decode(swh_ocean), 'k', lw=3)
plt.ylabel(swh_ocean.name + ' ['+swh_ocean.units+']', fontsize=15)
plt.xlabel('Along Track Samples', fontsize=15)
plt.show()

<span style='color:#ff6666'><font size="5">**Plot Exercise**

<font size="3"><span style='color:Black'> Choose a different variable, over a different time range, access a different OPeNDAP URL. Make sure to
adjust the:

- <font size="3"> longitudes.
- <font size="3"> latitudes.
- <font size="3"> standard name.
- <font size="3"> units.

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **You can inspect other data URL**


<font size="3.5"> `data_url2` is already defined above, but you can browse the catalog [here](https://cmr.earthdata.nasa.gov/virtual-directory/collections/C2799438313-POCLOUD/temporal).


In [ ]:
%%time
dataset2 = open_url(data_url2, session=my_session, protocol="dap4")

In [ ]:
dataset2.tree()